<a href="https://colab.research.google.com/github/Mustaq7777777/Introduction_To_Deep_Learning_DA6401/blob/main/Assignment1/DeepLearning_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

question 1

In [ ]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist


#There are 10 class labels present in mnist data set.
class_labels = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]


# wandb.init(project="DA6401-Assignment1", name="run1", reinit=True)

# #Loading data from the fashion_mnist
# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# #storing images in Image_set
# images_to_log = []
# for class_index in range(10):

#     #Find the instance of each class and append it .
#     idx = np.where(train_labels == class_index)[0][0]


#     image = train_images[idx]
#     wandb_image = wandb.Image(image, caption=class_labels[class_index])
#     images_to_log.append(wandb_image)

# # Log the image to WandB
# wandb.log({"Question1 Sample images for each class": images_to_log})


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

Data Preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)/256
x_test = x_test.reshape(10000, 784)/256

y_train = y_train.reshape(60000, 1)
y_test = y_test.reshape(10000, 1)

Activation Functions and their derivatives

In [ ]:

def softmax(v):
    exp_vector = np.exp(v - np.max(v))
    return exp_vector / np.sum(exp_vector)

# Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Tanh
def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - np.tanh(x) ** 2

# ReLU
def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    return np.where(x > 0, 1, 0)

# One-hot vector
def e(l, length):
    y = np.zeros([length, 1])
    y[int(l)] = 1
    return y

Initialization Of Weights and Biases

In [ ]:

def init(W, b, input_nodes, hiddenlayers, hiddennodes, output_nodes, initializer):

    np.random.seed(1)
    if initializer == "Xavier":
        # Xavier initialization for the first hidden layer (input to first hidden layer)
        W[1] = np.random.normal(0.0, np.sqrt(1.0 / input_nodes), (hiddennodes, input_nodes))
        b[1] = np.zeros((hiddennodes, 1))
        # Xavier initialization for subsequent hidden layers
        for i in range(2, hiddenlayers + 1):
            W[i] = np.random.normal(0.0, np.sqrt(1.0 / hiddennodes), (hiddennodes, hiddennodes))
            b[i] = np.zeros((hiddennodes, 1))
        # Xavier initialization for the output layer (from last hidden layer to output)
        W[hiddenlayers + 1] = np.random.normal(0.0, np.sqrt(1.0 / hiddennodes), (output_nodes, hiddennodes))
        b[hiddenlayers + 1] = np.zeros((output_nodes, 1))
    elif initializer == "random":
        # Random initialization for the first hidden layer (input to first hidden layer)
        W[1] = np.random.rand(hiddennodes, input_nodes) - 0.5
        b[1] = np.zeros((hiddennodes, 1))
        # Random initialization for subsequent hidden layers
        for i in range(2, hiddenlayers + 1):
            W[i] = np.random.rand(hiddennodes, hiddennodes) - 0.5
            b[i] = np.zeros((hiddennodes, 1))
        # Random initialization for the output layer (from last hidden layer to output)
        W[hiddenlayers + 1] = np.random.rand(output_nodes, hiddennodes) - 0.5
        b[hiddenlayers + 1] = np.zeros((output_nodes, 1))

    return W, b



Forward Propagation

In [ ]:
# For applying activation function
def g(z, act_func):
    if act_func == "sigmoid":
        return sigmoid(z)
    elif act_func == "tanh":
        return tanh(z)
    elif act_func == "relu":
        return relu(z)

# For applying derivative of activation function
def g_prime(z, act_func):
    if act_func == "sigmoid":
        return d_sigmoid(z)
    elif act_func == "tanh":
        return d_tanh(z)
    elif act_func == "relu":
        return d_relu(z)

# For calculating output using softmax
def o(z, act_func):
    if act_func == "softmax":
        return softmax(z)

# Applying forward propagation for the L-th training example
def forward_propagation(x_data, hidden_layers, hidden_nodes, input_nodes, output_nodes, W, b, a, h, L, act_func):
    h[0] = x_data[L].reshape(-1, 1)
    for layer in range(1, hidden_layers + 1):
        a[layer] = np.dot(W[layer], h[layer - 1]) + b[layer]
        h[layer] = g(a[layer], act_func)
    a[hidden_layers + 1] = np.dot(W[hidden_layers + 1], h[hidden_layers]) + b[hidden_layers + 1]
    ycap = o(a[hidden_layers + 1], "softmax")
    return ycap, a, h


Back Propagation

In [ ]:
def back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                     W, b, a, h, VW, Vb, sample_idx, yhat, activation_func, loss_func):

    output_layer = hiddenlayers + 1
    delta = {}

    # delta[outputlayer] computation based on loss function
    if loss_func == "cross_entropy":
        delta[output_layer] = yhat - e(y_train[sample_idx].item(), output_nodes)
    elif loss_func == "mean_squared_error":
        target = e(y_train[sample_idx].item(), output_nodes)
        # Compute the Jacobian matrix of the softmax output.
        jacobian = np.diagflat(yhat) - np.dot(yhat, yhat.T)
        # Apply the chain rule and scale by the number of output nodes (for the mean).
        delta[output_layer] = np.dot(jacobian, (yhat - target)) / output_nodes

    # propagating backwards by computing derivatives at each layer till the input layer
    for k in range(output_layer, 0, -1):
        grad_W = np.dot(delta[k], h[k-1].T)
        grad_b = delta[k]
        if k in VW:
            VW[k] += grad_W
            Vb[k] += grad_b
        else:
            VW[k] = grad_W
            Vb[k] = grad_b
        if k > 1:
            prev_error = np.dot(W[k].T, delta[k])
            delta[k-1] = np.multiply(prev_error, g_prime(a[k-1], activation_func).reshape(-1, 1))
    return VW, Vb

Loss Function

In [ ]:
# Loss function (for cross-entropy and mean_squared_error)
def loss(yhat, y_train, loss_func, sample_idx, output_nodes):
    if loss_func == "cross_entropy":
        return -np.log(yhat[y_train[sample_idx].item()])
    elif loss_func == "mean_squared_error":
        target = e(y_train[sample_idx].item(), output_nodes)
        return np.mean(np.square(yhat - target))

Evaluating Performance and Plotting

In [ ]:
def evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                         hidden_layers, hidden_nodes, input_nodes, output_nodes,
                         activation_func, loss_func):
    # Test accuracy.
    correct_test = 0
    for idx in range(len(x_test)):
        yhat, _, _ = forward_propagation(x_test, hidden_layers, hidden_nodes, input_nodes,
                                         output_nodes, W, b, {}, {}, idx, activation_func)
        pred = int(np.argmax(yhat))
        if pred == y_test[idx]:
            correct_test += 1
    test_accuracy = (correct_test / len(x_test)) * 100
    wandb.log({"test_accuracy": test_accuracy})

    # Validation on last 6000 training samples.
    correct_val = 0
    total_val_loss = 0
    for idx in range(54000, 60000):
        yhat, _, _ = forward_propagation(x_train, hidden_layers, hidden_nodes, input_nodes,
                                         output_nodes, W, b, {}, {}, idx, activation_func)
        total_val_loss += loss(yhat, y_train, loss_func, idx, output_nodes)
        pred = int(np.argmax(yhat))
        if pred == y_train[idx]:
            correct_val += 1
    val_accuracy = (correct_val / (60000 - 54000)) * 100
    val_loss = total_val_loss / (60000 - 54000)
    wandb.log({"val_accuracy": val_accuracy, "val_loss": val_loss})

    # Training on first 54000 samples.
    correct_train = 0
    total_train_loss = 0
    for idx in range(0, 54000):
        yhat, _, _ = forward_propagation(x_train, hidden_layers, hidden_nodes, input_nodes,
                                         output_nodes, W, b, {}, {}, idx, activation_func)
        total_train_loss += loss(yhat, y_train, loss_func, idx, output_nodes)
        pred = int(np.argmax(yhat))
        if pred == y_train[idx]:
            correct_train += 1
    train_accuracy = (correct_train / 54000) * 100
    train_loss = total_train_loss / 54000
    wandb.log({"train_accuracy": train_accuracy, "train_loss": train_loss})

    return train_accuracy, val_accuracy, test_accuracy

Stochastic Gradient Descent

In [ ]:
def SGD(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
        W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
        activation_func, loss_func, weight_decay):

    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            yhat, a, h = forward_propagation(x_train, hiddenlayers, hiddennodes, input_nodes,
                                             output_nodes, W, b, a, h, L, activation_func)
            curr_loss = loss(yhat, y_train, loss_func, L, output_nodes)
            total_loss += curr_loss
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W, b, a, h, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                for i in range(1, hiddenlayers + 2):
                    W[i] -= learning_rate * (VW[i] + weight_decay * W[i])
                    b[i] -= learning_rate * Vb[i]
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

Momentum Based Gradient Descent

In [ ]:
def momentum_gd(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
                activation_func, loss_func, weight_decay, momentum):
    vW = {}
    vB = {}
    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            yhat, a, h = forward_propagation(x_train, hiddenlayers, hiddennodes, input_nodes,
                                             output_nodes, W, b, a, h, L, activation_func)
            total_loss += loss(yhat, y_train, loss_func, L, output_nodes)
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W, b, a, h, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                for i in range(1, hiddenlayers + 2):
                    if i not in vW:
                        vW[i] = np.zeros_like(W[i])
                        vB[i] = np.zeros_like(b[i])
                    vW[i] = momentum * vW[i] + learning_rate * (VW[i] + weight_decay * W[i])
                    vB[i] = momentum * vB[i] + learning_rate * Vb[i]
                    W[i] -= vW[i]
                    b[i] -= vB[i]
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

Nesterov

In [ ]:
def nesterov_gd(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
                activation_func, loss_func, weight_decay, momentum):
    vW = {}
    vB = {}
    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            W_temp = {}
            b_temp = {}
            for i in range(1, hiddenlayers + 2):
                if i not in vW:
                    vW[i] = np.zeros_like(W[i])
                    vB[i] = np.zeros_like(b[i])
                W_temp[i] = W[i] - momentum * vW[i]
                b_temp[i] = b[i] - momentum * vB[i]
            yhat, a_temp, h_temp = forward_propagation(x_train, hiddenlayers, hiddennodes, input_nodes,
                                                       output_nodes, W_temp, b_temp, a, h, L, activation_func)
            total_loss += loss(yhat, y_train, loss_func, L, output_nodes)
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W_temp, b_temp, a_temp, h_temp, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                for i in range(1, hiddenlayers + 2):
                    vW[i] = momentum * vW[i] + learning_rate * (VW[i] + weight_decay * W[i])
                    vB[i] = momentum * vB[i] + learning_rate * Vb[i]
                    W[i] -= vW[i]
                    b[i] -= vB[i]
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

RMSprop

In [ ]:
def rmsprop(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
            W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
            activation_func, loss_func, weight_decay, beta, epsilon):
    sW = {}
    sB = {}
    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            yhat, a_temp, h_temp = forward_propagation(x_train, hiddenlayers, hiddennodes,
                                                       input_nodes, output_nodes, W, b, a, h, L, activation_func)
            total_loss += loss(yhat, y_train, loss_func, L, output_nodes)
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W, b, a, h, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                for i in range(1, hiddenlayers + 2):
                    if i not in sW:
                        sW[i] = np.zeros_like(W[i])
                        sB[i] = np.zeros_like(b[i])
                    grad_W = VW[i] + weight_decay * W[i]
                    grad_b = Vb[i]
                    sW[i] = beta * sW[i] + (1 - beta) * np.square(grad_W)
                    sB[i] = beta * sB[i] + (1 - beta) * np.square(grad_b)
                    W[i] -= learning_rate * grad_W / (np.sqrt(sW[i]) + epsilon)
                    b[i] -= learning_rate * grad_b / (np.sqrt(sB[i]) + epsilon)
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

Adam

In [ ]:
def adam(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
         W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
         activation_func, loss_func, weight_decay, beta1, beta2, epsilon):
    mW = {}
    vW = {}
    mB = {}
    vB = {}
    t = 0
    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            yhat, a_temp, h_temp = forward_propagation(x_train, hiddenlayers, hiddennodes,
                                                       input_nodes, output_nodes, W, b, a, h, L, activation_func)
            total_loss += loss(yhat, y_train, loss_func, L, output_nodes)
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W, b, a, h, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                t += 1
                for i in range(1, hiddenlayers + 2):
                    if i not in mW:
                        mW[i] = np.zeros_like(W[i])
                        vW[i] = np.zeros_like(W[i])
                        mB[i] = np.zeros_like(b[i])
                        vB[i] = np.zeros_like(b[i])
                    grad_W = VW[i] + weight_decay * W[i]
                    grad_b = Vb[i]
                    mW[i] = beta1 * mW[i] + (1 - beta1) * grad_W
                    mB[i] = beta1 * mB[i] + (1 - beta1) * grad_b
                    vW[i] = beta2 * vW[i] + (1 - beta2) * np.square(grad_W)
                    vB[i] = beta2 * vB[i] + (1 - beta2) * np.square(grad_b)
                    mW_hat = mW[i] / (1 - beta1**t)
                    mB_hat = mB[i] / (1 - beta1**t)
                    vW_hat = vW[i] / (1 - beta2**t)
                    vB_hat = vB[i] / (1 - beta2**t)
                    W[i] -= learning_rate * mW_hat / (np.sqrt(vW_hat) + epsilon)
                    b[i] -= learning_rate * mB_hat / (np.sqrt(vB_hat) + epsilon)
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

Nadam

In [ ]:
def nadam(x_train, y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
          W, b, a, h, learning_rate, num_epochs, batch_size, train_set,
          activation_func, loss_func, weight_decay, beta1, beta2, epsilon):
    mW = {}
    vW = {}
    mB = {}
    vB = {}
    t = 0
    for epoch in range(num_epochs):
        VW = {}
        Vb = {}
        total_loss = 0
        for L in range(train_set):
            yhat, a_temp, h_temp = forward_propagation(x_train, hiddenlayers, hiddennodes,
                                                       input_nodes, output_nodes, W, b, a, h, L, activation_func)
            total_loss += loss(yhat, y_train, loss_func, L, output_nodes)
            back_propagation(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
                             W, b, a, h, VW, Vb, L, yhat, activation_func, loss_func)
            if L % batch_size == 0:
                t += 1
                for i in range(1, hiddenlayers + 2):
                    if i not in mW:
                        mW[i] = np.zeros_like(W[i])
                        vW[i] = np.zeros_like(W[i])
                        mB[i] = np.zeros_like(b[i])
                        vB[i] = np.zeros_like(b[i])
                    grad_W = VW[i] + weight_decay * W[i]
                    grad_b = Vb[i]
                    mW[i] = beta1 * mW[i] + (1 - beta1) * grad_W
                    mB[i] = beta1 * mB[i] + (1 - beta1) * grad_b
                    vW[i] = beta2 * vW[i] + (1 - beta2) * np.square(grad_W)
                    vB[i] = beta2 * vB[i] + (1 - beta2) * np.square(grad_b)
                    mW_hat = mW[i] / (1 - beta1**t)
                    mB_hat = mB[i] / (1 - beta1**t)
                    vW_hat = vW[i] / (1 - beta2**t)
                    vB_hat = vB[i] / (1 - beta2**t)
                    nadam_update_W = (beta1 * mW_hat) + ((1 - beta1) * grad_W) / (1 - beta1**t)
                    nadam_update_B = (beta1 * mB_hat) + ((1 - beta1) * grad_b) / (1 - beta1**t)
                    W[i] -= learning_rate * nadam_update_W / (np.sqrt(vW_hat) + epsilon)
                    b[i] -= learning_rate * nadam_update_B / (np.sqrt(vB_hat) + epsilon)
                VW = {}
                Vb = {}
        wandb.log({"epoch": epoch, "loss": total_loss})
        evaluate_performance(x_test, y_test, x_train, y_train, W, b,
                             hiddenlayers, hiddennodes, input_nodes, output_nodes, activation_func, loss_func)

Training

In [ ]:
def train_model(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
                W, b, a, h, lr, epochs, batch, n_train, opt, act_func, loss_func, init_method,
                weight_decay, momentum, beta1, beta2, epsilon, beta):
    # Initialize weights and biases.
    init(W, b, input_nodes, hidden_layers, hidden_nodes, output_nodes, init_method)

    if opt == "sgd":
        SGD(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
            W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay)
    elif opt == "momentum":
        momentum_gd(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
                    W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay, momentum)
    elif opt == "nag":
        nesterov_gd(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
                     W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay, momentum)
    elif opt == "rmsprop":
        rmsprop(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
                W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay, beta, epsilon)
    elif opt == "adam":
        adam(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
             W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay, beta1, beta2, epsilon)
    elif opt == "nadam":
        nadam(x_train, y_train, hidden_layers, hidden_nodes, input_nodes, output_nodes,
              W, b, a, h, lr, epochs, batch, n_train, act_func, loss_func, weight_decay, beta1, beta2, epsilon)
    else:
        raise ValueError("Optimizer option not recognized.")

    # Evaluate final performance after training.
    final_train_acc, final_val_acc, final_test_acc = evaluate_performance(
        x_test, y_test, x_train, y_train, W, b,
        hidden_layers, hidden_nodes, input_nodes, output_nodes, act_func, loss_func
    )
    return final_train_acc, final_val_acc, final_test_acc

Sweep Configuration

In [ ]:
# Sweep config for wandb plotting
sweep_config = {
    'name'  : "MustaqAhamed",
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'hiddenlayers': {
            'values': [3, 4, 5]
        },
        'num_epochs': {
            'values': [5, 10]
        },
        'hiddennodes': {
            'values': [32, 64, 128]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'initializer': {
            'values': ["random", "Xavier"]
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'opt': {
            'values': ["sgd", "mbgd", "nesterov","rmsprop", "adam", "nadam"]
        },
        'activation_func': {
            'values': ["sigmoid","tanh","relu"]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.5]
        }
    }
}

# sweep_id = wandb.sweep(sweep_config, project="DA6401-Assignment1")

Creating Network and Running Experiment

In [ ]:
# Initializing and setting training size
num_classes = 10
input_size = 784
n_train_samples = 54000

# Initialize parameter dictionaries
loss_mode = "cross_entropy"
weights = {}
biases = {}
h = {}
a = {}

def run_experiment():
    run = wandb.init(config=sweep_config)
    cfg = run.config
    run.name = "epochs {} hidden_layers {} hidden_size {} learning_rate {} opt {} batch_size {} init {} activation {} weight_decay {}".format(
        cfg.num_epochs, cfg.hiddenlayers, cfg.hiddennodes, cfg.learning_rate, cfg.opt, cfg.batch_size, cfg.initializer, cfg.activation_func, cfg.weight_decay
    )
    train_model(
        x_train, y_train,
        cfg.hiddenlayers, cfg.hiddennodes,
        input_size, num_classes,
        weights, biases, a, h,
        cfg.learning_rate, cfg.num_epochs, cfg.batch_size,
        n_train_samples, cfg.opt,
        cfg.activation_func, loss_mode, cfg.initializer, cfg.weight_decay
    )

wandb.agent("9orvwc9k", run_experiment,project="DA6401-Assignment1" ,  count=30 )
wandb.finish()